# NSE control setup with pyMOR

In [ ]:
from data_utils import (solve_steadystate_nse, get_stokes_solution, linearized_convection, 
                        writevp_paraview, collect_vtu_files, eva_quadterm)

import numpy as np
import os
import pickle
import scipy
import scipy.io as spio
import scipy.linalg as spla
import scipy.sparse as sps
import scipy.sparse.linalg as spsla

 ### Setup Parameters

In [ ]:
Re = 110          # Reynolds number
control = 'bc'    # 'dist' for distributed control, 'bc' for control via boundary conditions or None
level = 2         # discretization level (1, 2 or 3)
palpha = 1e-3     # penalty for Robin-boundary (only in control == 'bc' case)

# some strings for data storage
data_str = 'data/' + 'lvl_' + str(level) + ('_' + control if control is not None else '')
setup_str = data_str + '/re_' + str(Re) + ('_palpha_' + str(palpha) if control=='bc' else '')

if not os.path.exists(setup_str):
    os.makedirs(setup_str)

In [ ]:
mats = spio.loadmat(data_str + '/mats')

M = mats['M']
J = mats['J']
hmat = mats['H']
fv = mats['fv'] + 1./Re*mats['fv_diff'] + mats['fv_conv']
fp = mats['fp'] + mats['fp_div']
pcmat = mats['Cp']
vcmat = mats['Cv']
NV, NP = fv.shape[0], fp.shape[0]

if control == 'bc':
    A = 1./Re*mats['A'] + mats['L1'] + mats['L2'] + 1./palpha*mats['Arob']
    B = 1./palpha*mats['Brob']
else:
    A = 1./Re*mats['A'] + mats['L1'] + mats['L2']
    B = mats['B']
    # restrict to less dofs in the input
    NU = B.shape[1]
    B  = B[:, [0, NU//2]]

### Steady-state NSE solution and linearized convection

In [ ]:
if not os.path.isfile(setup_str + '/ss_nse_sol'):
    # compute steady-state nse solution as linearization point
    ss_nse_v, _ = solve_steadystate_nse(mats, Re, control, palpha=palpha)
    
    # compute linearized convection
    conv_mat = linearized_convection(mats['H'], ss_nse_v)

    scipy.io.savemat(setup_str + '/ss_nse_sol', {'ss_nse_v': ss_nse_v, 'conv_mat': conv_mat})

else:
    ss_nse_sol = scipy.io.loadmat(setup_str + '/ss_nse_sol')
    ss_nse_v, conv_mat = ss_nse_sol['ss_nse_v'], ss_nse_sol['conv_mat']

### Reduce linearization with pyMOR

In [ ]:
from pymor.models.iosys import StokesDescriptorModel
from pymor.reductors.h2 import GapIRKAReductor
from pymor.reductors.bt import LQGBTReductor
from pymor.operators.numpy import NumpyMatrixOperator
from pymor.algorithms.to_matrix import to_matrix

# MOR setup
method = 'GapIRKA'
rom_ord = 10
tol = 1e-2

rom_str = setup_str + '/roms/' + method + '_' + str(rom_ord) + '_' + str(tol)

if control == 'bc':
    Aop = NumpyMatrixOperator(-1./Re * mats['A'] - 1./palpha*mats['Arob'] - conv_mat)
else:
    Aop = NumpyMatrixOperator(-1./Re * mats['A'] - conv_mat)

Eop = NumpyMatrixOperator(M)
Gop = NumpyMatrixOperator(J.T)
Bop = NumpyMatrixOperator(B)
Cop = NumpyMatrixOperator(vcmat)
fom = StokesDescriptorModel(Aop, Gop, Bop, Cop, None, Eop)

if control is not None:
    if not os.path.isfile(rom_str):
        if not os.path.exists(setup_str + '/roms'):
            os.makedirs(setup_str + '/roms')

        if method == 'GapIRKA':
            reductor = GapIRKAReductor(fom)
            rom = reductor.reduce(rom_ord, tol=tol, conv_crit='sigma', projection='Eorth')
        else:
            reductor = LQGBTReductor(fom)
            rom = reductor.reduce(rom_ord, projection='biorth')

        with open(rom_str, 'wb') as rom_file:
            pickle.dump({'reductor': reductor, 'rom': rom}, rom_file)

    else:
        with open(rom_str, 'rb') as rom_file: 
            rom_dict = pickle.load(rom_file)

        rom = rom_dict['rom']
        reductor = rom_dict['reductor']

### Simulation parameters

In [ ]:
# parameters for time-stepping
t0 = 0.
tE = 8.
Nts = 2**12
DT = (tE-t0)/Nts
trange = np.linspace(t0, tE, Nts+1)

# files for results and visualization
if not os.path.exists(setup_str + '/results'):
    os.makedirs(setup_str + '/results')

poutlist = []
voutlist = []
vfile = lambda t : setup_str + '/results/v_t{0}.vtu'.format(t)
pfile = lambda t : setup_str + '/results/p_t{0}.vtu'.format(t)
vfilerel = lambda t : 'results/v_t{0}.vtu'.format(t)
pfilerel = lambda t : 'results/p_t{0}.vtu'.format(t)
vfilelist = []
pfilelist = []
strtojson = data_str + '/visualization.jsn'

### Define Control

In [ ]:
if control is None:
    def bbcu(ko_state):
        return np.zeros((NV, 1))
    
    def update_ko_state(ko_state, Cv, DT):
        return ko_state

else:
    Arom = to_matrix(rom.A, format='dense').real
    Brom = to_matrix(rom.B, format='dense').real
    Crom = to_matrix(rom.C, format='dense').real

    XCARE = spla.solve_continuous_are(Arom, Brom, Crom.T @ Crom, np.eye(Brom.shape[1]), balanced=False)
    XFARE = spla.solve_continuous_are(Arom.T, Crom.T, Brom @ Brom.T, np.eye(Crom.shape[0]), balanced=False)

    # define control based on Kalman observer state
    def bbcu(ko_state):
        uvec = -Brom.T @ XCARE @ ko_state
        return B @ uvec

    ko1_mat = Arom - XFARE @ Crom.T @ Crom - Brom @ Brom.T @ XCARE
    ko2_mat = XFARE @ Crom.T
    lu_piv = spla.lu_factor(np.eye(rom_ord) - DT * ko1_mat)

    Css = vcmat @ ss_nse_v

    # function that determines the next state of the Kalman observer via implicit euler step
    def update_ko_state(ko_state, Cv, DT):
        return spla.lu_solve(lu_piv, ko_state + DT * ko2_mat @ (Cv - Css))

### Time stepping

In [ ]:
# introduce small perturbation to steady-state solution as initial value
pert = fom.A.source.project_onto_subspace(fom.A.operator.source.ones(), trans=True).to_numpy().T
old_v = ss_nse_v + 1e-3 * pert

# initialize state for observer
ko_state = np.zeros((rom_ord, 1))

sysmat = sps.vstack([
             sps.hstack([M+DT*A, -J.T]), 
             sps.hstack([J, sps.csc_matrix((NP, NP))])
         ]).tocsc()
sysmati = spsla.factorized(sysmat)

for k, t in enumerate(trange):
    crhsv = M*old_v + DT*(fv - eva_quadterm(hmat, old_v) + bbcu(ko_state))
    crhs = np.vstack([crhsv, fp])
    vp_new = np.atleast_2d(sysmati(crhs.flatten())).T
    old_v = vp_new[:NV]
    p = vp_new[NV:]
    Cv = vcmat @ old_v

    poutlist.append((pcmat*p)[0][0])
    voutlist.append((Cv).flatten())
    
    ko_state = update_ko_state(ko_state, Cv, DT)
    if np.mod(k, round(Nts/64)) == 0:
        print('timestep {0:4d}/{1}, t={2:f}, |v|={3:e}'.format(k, Nts, t, np.linalg.norm(old_v)))
        writevp_paraview(velvec=old_v, pvec=p, vfile=vfile(t), pfile=pfile(t), strtojson=strtojson)
        vfilelist.append(vfilerel(t))
        pfilelist.append(pfilerel(t))

### For visualization run 'paraview v_results.pvd'

In [ ]:
# create the *_results.pvd files based on the .vtu files from the time stepping
collect_vtu_files(vfilelist, setup_str + '/v_results.pvd')
collect_vtu_files(pfilelist, setup_str + '/p_results.pvd')